In [1]:
# imports
import os
import numpy as np
import json
import pandas as pd
import time

In [2]:
# scan text for predefined terms

text = 'We use CNN for anomaly detection. As Convolutional Neural Networks are great for ML.'

def text_match(text):
    ml_terms = pd.read_csv('../data/patterns/ml_terms.csv')
    ml_slugs = ml_terms['Slug'].tolist()
    ml_slugs = [x for x in ml_slugs if str(x) != 'nan']
    ml_terms = ml_terms['Term'].tolist()
    #print(ml_slugs)

    contained_slugs = [x for x in ml_slugs if x in text]
    contained_terms = [x for x in ml_terms if x.lower() in text.lower()]
    
    #if 'CNN' or 'ANN' or 'RNN' in contained_slugs:
    #    contained_slugs.remove('NN')
    
    return contained_slugs + contained_terms

# test function
matches = text_match(text)
print(matches)

['ML', 'NN', 'CNN', 'neural network', 'Convolutional Neural Network', 'Anomaly detection']


In [3]:
# python code to package parser

fp1 = '../data/repositories/git/crowdAI/train-schedule-optimisation-challenge-starter-kit/utils/route_penalty.py'
fp2 = '../data/repositories/git/altosaar/food2vec/src/process_scraped_data.py'

def analyze_pythoncode(code):
    meta = {}
    debug = False
    
    #print(code)
    code = code.split('\n')
    
    libs = []
    keywords = []
    for line in code:
        line = line.strip()
        
        # get libraries
        words = line.split(' ')
        if words[0] == 'import' or words[0] == 'from':
            if debug: print('raw: ' + line.strip())
            lib = line.split(' ')
            if len(lib) > 1:
                lib = lib[1].split('.')[0].strip()
            else:
                break
            if ',' in lib:
                lib = [elem.strip() for elem in lib.split(',')]
                libs.extend(lib)
                if debug: print('parsed: ['+', '.join(lib) + ']')
            else:
                libs.append(lib)
                if debug: print('parsed: '+lib)
                    
        # analyze comments
        if len(line) > 0 and line[0] == '#':
            matches = text_match(line)
            keywords.extend(matches)
            if debug: print('keywords: '+keywords)

    # make values unique
    libs = np.unique(libs).tolist()
    meta['libs'] = libs
    
    keywords = np.unique(keywords).tolist()
    meta['keywords'] = keywords
    
    return meta

def analyze_py(filepath):
    meta = {}
    
    
    with open(filepath, encoding='utf-8', errors="ignore") as fp:
        content = fp.read()
        
    return analyze_pythoncode(content)
    #print(libs)

# test function
meta = analyze_py(fp2)
print(meta)

{'libs': ['json', 'os', 're', 'util'], 'keywords': []}


In [4]:
# Jupyter Notebook to python code parser

fp1 = '../data/repositories/git/Lemma1/DPFE/DPFE-v0.1.ipynb'
fp2 = '../data/repositories/git/A7med01/Deep-learning-for-Animal-Identification/Identify the Animal_colab.ipynb'
fp3 = '../data/repositories/git/abhiagar90/power_networks/djnagowork/tempsite/src/accumulate_todb/AllPartiesTODB.ipynb'
fp4 = '../data/repositories/git/everAspiring/RegressionAnalysis/RestaurantProfits.ipynb'

def analyze_ipynb(filepath):
    meta = {}
    py = ''
    debug = True
    cells = None
    
    with open(filepath, encoding='utf-8', errors="ignore") as fp:
        c = fp.read()
        try:
            c = json.loads(c)
            cells = c.get('cells')
        except:
            print('no json data')
        
        #if cells == None:
        #    worksheets = c.get('worksheets')
        #    cells = worksheets[0].get('cells')

        #print(cells[0])
            
    if not cells == None:
        for cell in cells:
            if cell.get('source'):
                for source in cell['source']:
                    #print(source.strip())
                    py += source
                py += '\n\n'

            if cell.get('input'):
                for source in cell['input']:
                    #print(source.strip())
                    py += source
                py += '\n\n'

        #print(py)
    return analyze_pythoncode(py)

# test function
meta = analyze_ipynb(fp1)
print(meta)

{'libs': ['base', 'collections', 'copy', 'datetime', 'joblib', 'matplotlib', 'networkx', 'numpy', 'os', 'pandas', 'pfe', 'pickle', 'random', 'scipy', 'seaborn'], 'keywords': []}


In [5]:
# python package pattern matching with limited set

def package_match_simple(libs):
    ml_libs = ['tensorflow','keras','pytorch','sklearn','nltk','mllib','theano','mxnet','lightgbm','scipy','caffe','cntk','auto_ml','OpenNN','h2o']
    matches = set(libs).intersection(ml_libs)
    if not bool(matches):
        matches = {}
    return matches

# test function
matches = package_match_simple(meta['libs'])
print(matches)

{'scipy'}


In [6]:
# python package pattern matching with extended set

def package_match(libs):
    ml_libs = pd.read_csv('../data/patterns/ml_libraries.csv')
    ml_packages = ml_libs['Python Package'].tolist()

    matches = set(libs).intersection(ml_packages)
    if not bool(matches):
        matches = {}
    return matches

# test function
matches = package_match(meta['libs'])
print(matches)

{}


In [7]:
# directory walker / run analyze-parser / store results

dir_base = '../data/repositories/git/'
dir1 = 'finnqiao\\cohort_online_retail\\'
dir2 = 'arvindkarir\\retail\\'
dir3 = 'crowdAI\\train-schedule-optimisation-challenge-starter-kit\\'
dir4 = 'Lemma1\\DPFE\\'
dir5 = 'LeadingIndiaAI\\Forest-Fire-Detection-through-UAV-imagery-using-CNNs\\'
dir6 = 'firmai\\financial-machine-learning\\'

# helper function to create folder create_folder
def create_folder(path):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
            print(path + ' created')
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
                
def dir_walk(dir, store=True):
    meta = []
    dir_base = '../data/repositories/git/'
    path_meta = '/.meta\\git_analyzer_results.json'
    
    for root, dirs, files in os.walk(dir):
        #path = root.split(os.sep)
        #print((len(path) - 1) * '-', os.path.basename(root))
        
        for file in files:
            #print(len(path) * '-', file)
            file_meta = {}
            
            if '.ipynb' in file:
                print('ipynb: '+root+'/'+file)
                file_meta = analyze_ipynb(os.path.join(root, file))
                
            if '.py' in file:
                print('py: '+root+'/'+file)
                file_meta = analyze_py(os.path.join(root, file))
                
            if '.md' in file:
                print('md: '+root+'/'+file)
                with open(os.path.join(root, file), encoding='utf-8', errors="ignore") as fp:
                    content = fp.read()
                matches = text_match(content)
                if bool(matches):
                    file_meta['keywords'] = matches
                
            if bool(file_meta):
                relative = os.path.relpath(root, dir)
                file = relative+'\\'+file
                print('relative: '+file)
                
                if 'libs' in file_meta:
                    file_meta['ml_libs'] = list(package_match(file_meta['libs']))
                
                meta.append({'file': file, 'meta': file_meta})

    if store:
        path_meta = os.path.join(dir+path_meta)
        print('meta: '+path_meta)
        #create_folder(path_meta)
        with open(path_meta, 'w') as fp:
            fp.write(json.dumps(meta))
    
    return meta
                
meta = dir_walk(dir_base+dir6)
print(meta)
#print(json.dumps(meta))

md: ../data/repositories/git/firmai\financial-machine-learning\/README.md
relative: .\README.md
meta: ../data/repositories/git/firmai\financial-machine-learning\/.meta\git_analyzer_results.json
[{'file': '.\\README.md', 'meta': {'keywords': ['ML', 'AI', 'NN', 'DL', 'NLP', 'RL', 'machine learning', 'neural network', 'predict', 'regression', 'classification', 'model', 'layer', 'deep learning', 'Recurrent Neural Network', 'autoencoder', 'Bayesian', 'Sentiment Analysis', 'clustering', 'Reinforcement learning', 'Reinforcement learning', 'classification', 'Regression', 'Clustering', 'LSTM', 'GRU']}}]


In [8]:
dir_base = '../data/repositories/git/'

authors = os.listdir(dir_base)

a = 0
p = 0

start = 0
end = 1000

runtime_start = time.time()

i = start
for author in authors:
    if os.path.isdir(os.path.join(dir_base, author)):
        
        projects = os.listdir(os.path.join(dir_base, author))
        #print(author, projects)
        for project in projects:
            #print(project)
            if p >= start:
                if os.path.isdir(os.path.join(dir_base, author, project)):
                    print('### ' + str(i) + ': ' + author, project)
                    meta = dir_walk(os.path.join(dir_base, author, project))
                    #print(meta)
                    
                i += 1

            # count projects
            p += 1
        #count authors
        a += 1
        
    # break loop
    if i>=end and True:
        break
    
runtime_end = time.time()
print('### parsed {0} git-projects  in {1} seconds ###'.format(i-start, round(runtime_end - runtime_start, 3)))

### 0: 1davegalloway SchoolDistrictAnalysis
py: ../data/repositories/git/1davegalloway\SchoolDistrictAnalysis/DavesPandadHW.py
relative: .\DavesPandadHW.py
ipynb: ../data/repositories/git/1davegalloway\SchoolDistrictAnalysis/PyCitySchools_starter.ipynb
relative: .\PyCitySchools_starter.ipynb
md: ../data/repositories/git/1davegalloway\SchoolDistrictAnalysis/README.md
ipynb: ../data/repositories/git/1davegalloway\SchoolDistrictAnalysis/Sch.District.Analysis.ipynb
relative: .\Sch.District.Analysis.ipynb
ipynb: ../data/repositories/git/1davegalloway\SchoolDistrictAnalysis\.ipynb_checkpoints/PyCitySchools_starter-checkpoint.ipynb
relative: .ipynb_checkpoints\PyCitySchools_starter-checkpoint.ipynb
ipynb: ../data/repositories/git/1davegalloway\SchoolDistrictAnalysis\.ipynb_checkpoints/Sch.District.Analysis-checkpoint.ipynb
relative: .ipynb_checkpoints\Sch.District.Analysis-checkpoint.ipynb
meta: ../data/repositories/git/1davegalloway\SchoolDistrictAnalysis/.meta\git_analyzer_results.json
### 

relative: djnagowork\tempsite\src\accumulate_todb\checkpoint.py
ipynb: ../data/repositories/git/abhiagar90\power_networks\djnagowork\tempsite\src\accumulate_todb/Constituencies_Final_Actual.ipynb
relative: djnagowork\tempsite\src\accumulate_todb\Constituencies_Final_Actual.ipynb
py: ../data/repositories/git/abhiagar90\power_networks\djnagowork\tempsite\src\accumulate_todb/csv2gdb.py
relative: djnagowork\tempsite\src\accumulate_todb\csv2gdb.py
py: ../data/repositories/git/abhiagar90\power_networks\djnagowork\tempsite\src\accumulate_todb/gdb2csv.py
relative: djnagowork\tempsite\src\accumulate_todb\gdb2csv.py
ipynb: ../data/repositories/git/abhiagar90\power_networks\djnagowork\tempsite\src\accumulate_todb/IndianStates_AfterChange_Latest.ipynb
relative: djnagowork\tempsite\src\accumulate_todb\IndianStates_AfterChange_Latest.ipynb
ipynb: ../data/repositories/git/abhiagar90\power_networks\djnagowork\tempsite\src\accumulate_todb/NationalPartiesToDB.ipynb
relative: djnagowork\tempsite\src\accu

py: ../data/repositories/git/abhiagar90\power_networks\djnagowork\tempsite\src\utils/db2csv.py
relative: djnagowork\tempsite\src\utils\db2csv.py
ipynb: ../data/repositories/git/abhiagar90\power_networks\djnagowork\tempsite\src\utils/hashlib_usage.ipynb
relative: djnagowork\tempsite\src\utils\hashlib_usage.ipynb
ipynb: ../data/repositories/git/abhiagar90\power_networks\djnagowork\tempsite\src\utils/LevenshteinFuzzyPhonetic.ipynb
relative: djnagowork\tempsite\src\utils\LevenshteinFuzzyPhonetic.ipynb
py: ../data/repositories/git/abhiagar90\power_networks\djnagowork\tempsite\src\utils/test.py
relative: djnagowork\tempsite\src\utils\test.py
ipynb: ../data/repositories/git/abhiagar90\power_networks\djnagowork\tempsite\src\utils/Utilities.ipynb
relative: djnagowork\tempsite\src\utils\Utilities.ipynb
ipynb: ../data/repositories/git/abhiagar90\power_networks\djnagowork\tempsite\src\utils/WinnersEditDistnaceBusinness.ipynb
relative: djnagowork\tempsite\src\utils\WinnersEditDistnaceBusinness.ipyn

md: ../data/repositories/git/abhiagar90\power_networks\nexus\app\static\js\plugins\sigma.helpers.graph/README.md
md: ../data/repositories/git/abhiagar90\power_networks\nexus\app\static\js\plugins\sigma.layouts.dagre/README.md
relative: nexus\app\static\js\plugins\sigma.layouts.dagre\README.md
md: ../data/repositories/git/abhiagar90\power_networks\nexus\app\static\js\plugins\sigma.layouts.forceAtlas2/README.md
md: ../data/repositories/git/abhiagar90\power_networks\nexus\app\static\js\plugins\sigma.layouts.forceLink/README.md
relative: nexus\app\static\js\plugins\sigma.layouts.forceLink\README.md
md: ../data/repositories/git/abhiagar90\power_networks\nexus\app\static\js\plugins\sigma.layouts.fruchtermanReingold/README.md
md: ../data/repositories/git/abhiagar90\power_networks\nexus\app\static\js\plugins\sigma.layouts.noverlap/README.md
md: ../data/repositories/git/abhiagar90\power_networks\nexus\app\static\js\plugins\sigma.parsers.cypher/README.md
relative: nexus\app\static\js\plugins\sig

py: ../data/repositories/git/abhiagar90\power_networks\nexusapis\app/__init__.py
relative: nexusapis\app\__init__.py
py: ../data/repositories/git/abhiagar90\power_networks\nexusapis\app\newlinks/temp.py
relative: nexusapis\app\newlinks\temp.py
py: ../data/repositories/git/abhiagar90\power_networks\nexusapis\app\newlinks/utils_crawler.py
relative: nexusapis\app\newlinks\utils_crawler.py
py: ../data/repositories/git/abhiagar90\power_networks\nexusapis\app\newlinks/validations.py
relative: nexusapis\app\newlinks\validations.py
py: ../data/repositories/git/abhiagar90\power_networks\nexusapis\app\newlinks/views.py
relative: nexusapis\app\newlinks\views.py
py: ../data/repositories/git/abhiagar90\power_networks\nexusapis\app\newlinks/__init__.py
relative: nexusapis\app\newlinks\__init__.py
ipynb: ../data/repositories/git/abhiagar90\power_networks\nexusapis\tmp/RestAPIWork.ipynb
relative: nexusapis\tmp\RestAPIWork.ipynb
ipynb: ../data/repositories/git/abhiagar90\power_networks\nexusapis\tmp/Un

...

relative: .\CrimePrediction.ipynb
md: ../data/repositories/git/vikram-bhati\PAASBAAN-crime-prediction/README.md
relative: .\README.md
meta: ../data/repositories/git/vikram-bhati\PAASBAAN-crime-prediction/.meta\git_analyzer_results.json
### 420: Vipul115 Statistical-Time-Series-Analysis-on-Agricultural-Commodity-Prices
md: ../data/repositories/git/Vipul115\Statistical-Time-Series-Analysis-on-Agricultural-Commodity-Prices/README.md
relative: .\README.md
ipynb: ../data/repositories/git/Vipul115\Statistical-Time-Series-Analysis-on-Agricultural-Commodity-Prices/Task 2  (pre-processing).ipynb
relative: .\Task 2  (pre-processing).ipynb
ipynb: ../data/repositories/git/Vipul115\Statistical-Time-Series-Analysis-on-Agricultural-Commodity-Prices/Task 3 Compare prices.ipynb
relative: .\Task 3 Compare prices.ipynb
ipynb: ../data/repositories/git/Vipul115\Statistical-Time-Series-Analysis-on-Agricultural-Commodity-Prices/Task- 2 Account Seasonality.ipynb
relative: .\Task- 2 Account Seasonality.ipynb
i

meta: ../data/repositories/git/widdowquinn\Teaching-EMBL-Plant-Path-Genomics/.meta\git_analyzer_results.json
### 429: williamadams1 natural-gas-consumption-forecasting
ipynb: ../data/repositories/git/williamadams1\natural-gas-consumption-forecasting/Forecasting Residential US Natural Gas Consumption.ipynb
relative: .\Forecasting Residential US Natural Gas Consumption.ipynb
md: ../data/repositories/git/williamadams1\natural-gas-consumption-forecasting/README.md
relative: .\README.md
meta: ../data/repositories/git/williamadams1\natural-gas-consumption-forecasting/.meta\git_analyzer_results.json
### 430: worldbank ML-classification-algorithms-poverty
md: ../data/repositories/git/worldbank\ML-classification-algorithms-poverty/README.md
relative: .\README.md
md: ../data/repositories/git/worldbank\ML-classification-algorithms-poverty\competition_winning_models_malawi\malawi\1st Place/Model documentation and write-up.md
relative: competition_winning_models_malawi\malawi\1st Place\Model docume

ipynb: ../data/repositories/git/worldbank\ML-classification-algorithms-poverty\notebooks/00.1-new-idn-data-preparation.ipynb
relative: notebooks\00.1-new-idn-data-preparation.ipynb
ipynb: ../data/repositories/git/worldbank\ML-classification-algorithms-poverty\notebooks/01.0-lr-mwi.ipynb
relative: notebooks\01.0-lr-mwi.ipynb
ipynb: ../data/repositories/git/worldbank\ML-classification-algorithms-poverty\notebooks/01.1-lr-idn.ipynb
relative: notebooks\01.1-lr-idn.ipynb
ipynb: ../data/repositories/git/worldbank\ML-classification-algorithms-poverty\notebooks/02.0-lda-mwi.ipynb
relative: notebooks\02.0-lda-mwi.ipynb
ipynb: ../data/repositories/git/worldbank\ML-classification-algorithms-poverty\notebooks/02.1-lda-idn.ipynb
relative: notebooks\02.1-lda-idn.ipynb
ipynb: ../data/repositories/git/worldbank\ML-classification-algorithms-poverty\notebooks/03.0-knn-mwi.ipynb
relative: notebooks\03.0-knn-mwi.ipynb
ipynb: ../data/repositories/git/worldbank\ML-classification-algorithms-poverty\notebooks

md: ../data/repositories/git/xinychen\transdim\datasets/README.md
ipynb: ../data/repositories/git/xinychen\transdim\experiments/Imputation-BayesTRMF.ipynb
relative: experiments\Imputation-BayesTRMF.ipynb
ipynb: ../data/repositories/git/xinychen\transdim\experiments/Imputation-BayesTRTF.ipynb
relative: experiments\Imputation-BayesTRTF.ipynb
ipynb: ../data/repositories/git/xinychen\transdim\experiments/Imputation-BGCP.ipynb
relative: experiments\Imputation-BGCP.ipynb
ipynb: ../data/repositories/git/xinychen\transdim\experiments/Imputation-BPMF.ipynb
relative: experiments\Imputation-BPMF.ipynb
ipynb: ../data/repositories/git/xinychen\transdim\experiments/Imputation-BPTF.ipynb
relative: experiments\Imputation-BPTF.ipynb
ipynb: ../data/repositories/git/xinychen\transdim\experiments/Imputation-BTMF.ipynb
relative: experiments\Imputation-BTMF.ipynb
ipynb: ../data/repositories/git/xinychen\transdim\experiments/Imputation-BTTF.ipynb
relative: experiments\Imputation-BTTF.ipynb
ipynb: ../data/rep

ipynb: ../data/repositories/git/zhentaoshi\econ5170\Python/Lecture_2.ipynb
relative: Python\Lecture_2.ipynb
ipynb: ../data/repositories/git/zhentaoshi\econ5170\Python/Lecture_3.ipynb
relative: Python\Lecture_3.ipynb
ipynb: ../data/repositories/git/zhentaoshi\econ5170\Python/Lecture_4.ipynb
relative: Python\Lecture_4.ipynb
ipynb: ../data/repositories/git/zhentaoshi\econ5170\Python/Lecture_6.ipynb
relative: Python\Lecture_6.ipynb
md: ../data/repositories/git/zhentaoshi\econ5170\Python/note.md
py: ../data/repositories/git/zhentaoshi\econ5170\Python/Poisson.py
relative: Python\Poisson.py
py: ../data/repositories/git/zhentaoshi\econ5170\Python/summ.py
relative: Python\summ.py
py: ../data/repositories/git/zhentaoshi\econ5170\Python/unit_root_sim.py
relative: Python\unit_root_sim.py
ipynb: ../data/repositories/git/zhentaoshi\econ5170\Python/ZT_lec1-R.ipynb
relative: Python\ZT_lec1-R.ipynb
meta: ../data/repositories/git/zhentaoshi\econ5170/.meta\git_analyzer_results.json
### 437: zischwartz wo